以下の設定になっていること  

ランタイム > ランタイムのタイプを変更  
ハードウェアアクセラレータ：GPU  
ランタイムの仕様：ハイメモリ  

編集 > ノートブックの設定  
ハードウェアアクセラレータ：GPU  
ランタイムの仕様：ハイメモリ  

In [1]:
# googleドライブをマウント
from google.colab import drive 
drive.mount('/content/drive')
# 表示されるリンクをクリックして、アクセスを許可して、最後に表示される文字列を以下の入力欄に入れる

Mounted at /content/drive


In [2]:
# 作業フォルダに移動
%cd '/content/drive/My Drive/work/'

/content/drive/My Drive/work


In [3]:
# importで使う必要があるので、インストールがランタイム切れるごとに必要
# インストール後にランタイムの再起動を行わないとT5Tokenizerが見つからない
# メニュー「ランタイム → ランタイムを再起動」で「Google Colab」を再起動

# ドライブに保存してるものでインストール
!pip install -e transformers

# Huggingface Datasetsのインストール
!pip install datasets==1.2.1

# Sentencepieceのインストール
!pip install sentencepiece==0.1.91

Obtaining file:///content/drive/My%20Drive/work/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 3.3MB 6.6MB/s 
     |████████████████████████████████| 901kB 37.6MB/s 
  Running setup.py develop for transformers
     |████████████████████████████████| 163kB 6.8MB/s 
     |████████████████████████████████| 245kB 8.2MB/s 
     |████████████████████████████████| 1.1MB 4.9MB/s 


In [1]:
# 作業フォルダに移動
%cd '/content/drive/My Drive/work/'

/content/drive/My Drive/work


In [38]:
import json
path_w = 'train.txt'
with open(path_w, mode='w') as f_out:
  with open('output_0001.json') as f_in:
    json_text = json.load(f_in)
    for v_result in json_text.values():
      for v_alternatives in v_result:
        for val_list in v_alternatives.values():
          if isinstance(val_list, list):
            for val in val_list:
              # print(val.get('transcript'))
              f_out.write(val.get('transcript'))
              f_out.write('。')

CLM（Causal Language Modeling）: GPT、GPT-2

・model_name_or_path: モデルのチェックポイント（モデルを最初から学習しない場合）  
・model_type: モデルの種別（モデルを最初から学習する場合）  
・config_name: コンフィグ名（model_nameと同じでない場合）  
・tokenizer_name: トークナイザー名（model_nameと同じでない場合）  
・cache_dir: キャッシュフォルダ  
・use_fast_tokenizer: Fastトークナイザーを使用するかどうか  
・model_revision: 使用するモデルの特定のバージョン  
・use_auth_token: 「transformers-cli login」の実行時に生成されたトークンを使用するかどうか  

・dataset_name: データセット名  
・dataset_config_name: データセットのコンフィグ名  
・train_file: 学習データ（テキストファイル）  
・validation_file: 検証データ（テキストファイル）  
・overwrite_cache: キャッシュの上書き  
・validation_split_percentage: 学習データから使われる検証データの割合（検証データがない場合）  
・max_seq_length: トークン化後の最大合計入力シーケンス長  
・preprocessing_num_workers: 前処理に使用するプロセス数  
・block_size: トークン化後のオプションの入力シーケンス長  
・max_train_samples: 学習データの最大数  
・max_val_samples: 検証データの最大数  

    # GPT2のモデルファイルを指定
    --model_name_or_path=rinna/japanese-gpt2-medium \
    # 学習ファイル
    --train_file=train.txt \
    # 評価データファイル
    --validation_file=train.txt \
    # トレーニングを実施する
    --do_train \
    # 評価を実施する
    --do_eval \
    # 学習回数（エポック数）
    --num_train_epochs=30 \
    # チェックポイントの保存間隔
    --save_steps=5000 \
    # チェックポイントの保持数
    --save_total_limit=3 \
    # T5Tokenizer.model_max_length=1024をチャンクサイズとして使用するかblock_sizeで指定するかを設定する（設定しないとtokenizerの超大なサイズから1024になる）メモリに乗るように調整する必要がある（バッチサイズとの兼ね合い）
    --block_size=512 \
    # GPU1つあたりの学習バッチサイズ
    --per_device_train_batch_size=2 \
    # GPU1つあたりの評価バッチサイズ
    --per_device_eval_batch_size=2 \
    # モデルとチェックポイントの出力先
    --output_dir=output/ \
    # 出力先の上書きの許可
    --overwrite_output_dir=True \
    # T5Tokenizerで高速化ライブラリがあれば使用する
    --use_fast_tokenizer=False

### 学習する回数
エポック数 * (データ文字数 / ブロックサイズ):1つの学習データ / バッチサイズ  
    エポック数 = 1  
    データ文字数 = 22142  
    データの文字数だと数字が合わないので、tokenizeした結果の形態素数による？
    ブロックサイズ = 512  
    データ文字数 / ブロックサイズ = 43  
    バッチサイズ = 1  
    Total optimization steps = 29  


バッチサイズ = 2  
バッチサイズを2にすることで、トータルの実行回数が減っている  
Total optimization steps = 15  

In [42]:
%%time

# ファインチューニングの実行
!python ./transformers/examples/language-modeling/run_clm.py \
    --model_name_or_path=rinna/japanese-gpt2-medium \
    --train_file=train.txt \
    --do_train \
    --num_train_epochs=100 \
    --save_steps=5000 \
    --save_total_limit=3 \
    --block_size=256 \
    --per_device_train_batch_size=4 \
    --output_dir=output/ \
    --overwrite_output_dir=True \
    --use_fast_tokenizer=False

2021-06-28 15:11:30.908292: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
06/28/2021 15:11:32 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
06/28/2021 15:11:32 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=output/, overwrite_output_dir=True, do_train=True, do_eval=None, do_predict=False, evaluation_strategy=IntervalStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=4, per_device_eval_batch_size=8, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=5e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=100.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_ratio=0.0, warmup_steps=0, logging_dir=runs/Jun28_15-11-32_2d4200e574ad, logging_strategy=IntervalStrategy.STEPS, logging_first_step=False, l

In [43]:
from transformers import T5Tokenizer, AutoModelForCausalLM

# トークナイザーとモデルの準備
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-medium")
model = AutoModelForCausalLM.from_pretrained("output/")

In [44]:
# 推論
input = tokenizer.encode("青コーナー", return_tensors="pt")
output = model.generate(input, do_sample=True, max_length=100, num_return_sequences=8)
print(tokenizer.batch_decode(output))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


['青コーナー</s> 天井まで20m 青コーナーから3人が 降りてきました。 赤コーナーから2人が降りてきました。 青コーナーから2人 降りてきました。 青コーナーから2人 降りてきました。 赤コーナーから2人 降りてきました。 赤コーナーから1人 降りてきました。 赤コーナーから2人 [...] 青コーナーから2人 降りてきました。', '青コーナー</s> スタートです。左の茶道ていくらですか? 香水は数珠を持っていますか? 赤コーナー はい いいえ 無記名です。お点前しませんが ご苦労様でした。次はいらしてください。御席へとご案内します。お菓子をお出しします。手水鉢をお示ししますのでご自由にお飲みください。 青コーナー はい いいえ', '青コーナー</s> 雨にぬれても 濡れても 強くなれるよ 僕は負けない [dvd]出版社/メーカー: ポニーキャニオン■発売日: 2008/09/21■値段: <unk> 6,480(税込) ■発売場所: dvd・cdショップ ■商品紹介: ■青コーナー 雨にぬれても 濡れても 強くなれるよ 僕は負けない ■青コーナー 雨にぬれても', '青コーナー</s> 1 ○ 2 ○ 3 ○ 4 ○ 5 ○ 6 三十路コーナー 1 ○ 2 ○ 3 ○ 4 ○ 5 ○ 6 ○ 7 19 青コーナー 1 ○ 3 ○ 4 ○ 4 ○ 5 ○ 6 ○ 7 9 青コーナー 1 ○ 2 ○ 3 ○ 4 ○ 5 ○ 7 19 赤コーナー 1 ○ 2 ○', '青コーナー</s> 対 馬連4点 的中おめでとうございます。今年の函館2歳ステークスにふさわしいレースでした。来年も注目していきましょう。(大沼博) 函館2歳s 1〜4着まで馬券になっています。オッズの変動も大きいですね。ここぞという時にしっかり結果を出しています。来年の函館2歳sも注目していきましょう。(大沼博) 函館2歳s', '青コーナー</s> 何事もなかったかのように再開して、2連勝もあってリングに戻りましたが、やはりあの重量挙げまくりのシーンはなかなか見ていて気持ちよかったですね 武尊 そうですねあの時、手数を増やして配達に回ってたおかげでリングにもしっかりと顔を出してるし手数もどんどん出していったのでそれで勝負強いんだなというのを見せられたのでよかったですねあの試合からどれだけ